In [ ]:
%run /home/bb/src/python/valkyrie/apps/common/run_notebook_import.py

In [ ]:
import copy
from itertools import product as cartesian_product

from overrides import overrides
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms
from torch.nn import functional as F

from valkyrie.tools import *
from valkyrie.ml import modules
from valkyrie.ml.utils import tensor, HyperParameters
from valkyrie.ml import utils as ml_utils
from valkyrie.nibelungen.data import DataMgr, Df2T2

In [ ]:
from valkyrie.ml.data import DataModule

sdate, edate = '2023-01-01', '2023-01-31'
instr = 'BDM.BTC.USDT.FP'
freq = '1s'
ret_n_s = [30]

#########################################################
#DataMgr
#########################################################
data_mgr = DataMgr(sdate, edate, freq, '/home/bb/data/BDM', instrs = ['BDM.BTC.USDT.FP'], ret_n_s = [120])
df_res = data_mgr.get(instr) 
channels = ['bpx_last','apx_last','bq_last','aq_last','buy_qty_sum', 'sell_qty_sum']
n_channels = len(channels)
df2t2 = Df2T2(df_res, M = 16, xcols = channels, ycol = 'mid_last_ret_120_n')
dm = DataModule.from_dataset(df2t2, None, 2048)

In [ ]:
print_df_mem_usage(df_res)

In [ ]:
len(df2t2)

In [ ]:
def eval_init_loss(net, dm):
            
    init_loss, loss = 0.0, 0.0
    dl = dm.get_dataloader(True)
    for i, batch in enumerate(dl):
        print(i)
        print(batch[0].shape)
        X, YW = batch[0], batch[-1]                
        ZEROS = torch.zeros(YW.shape[0])
        with torch.no_grad():            
            init_loss += net.loss(ZEROS, YW)
            Y_hat = net(X.to(ml_utils.gpu()))
            loss += net.loss(Y_hat, YW.to(ml_utils.gpu()))
    return init_loss, loss

In [ ]:
class Inception(nn.Module):
    # c1--c4 are the number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)
        

    def forward(self, x):        
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

class GoogleNet(modules.Regressor):
    def b1(self):
        return nn.Sequential(            
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
    
@d2l.add_to_class(GoogleNet)
def b2(self):
    return nn.Sequential(        
        nn.LazyConv2d(64, kernel_size=1), nn.ReLU(),
        nn.LazyConv2d(192, kernel_size=3, padding=1), nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

@d2l.add_to_class(GoogleNet)
def b3(self):
    return nn.Sequential(        
        Inception(64, (96, 128), (16, 32), 32),
        Inception(128, (128, 192), (32, 96), 64),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

@d2l.add_to_class(GoogleNet)
def b4(self):
    return nn.Sequential(
                         Inception(192, (96, 208), (16, 48), 64),
                         Inception(160, (112, 224), (24, 64), 64),
                         Inception(128, (128, 256), (24, 64), 64),
                         Inception(112, (144, 288), (32, 64), 64),
                         Inception(256, (160, 320), (32, 128), 128),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

@d2l.add_to_class(GoogleNet)
def b4(self):
    return nn.Sequential(
                         Inception(192, (96, 208), (16, 48), 64),
                         Inception(160, (112, 224), (24, 64), 64),
                         Inception(128, (128, 256), (24, 64), 64),
                         Inception(112, (144, 288), (32, 64), 64),
                         Inception(256, (160, 320), (32, 128), 128),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

@d2l.add_to_class(GoogleNet)
def b5(self):
    return nn.Sequential(
                         Inception(256, (160, 320), (32, 128), 128),
                         Inception(384, (192, 384), (48, 128), 128),
                         nn.AdaptiveAvgPool2d((1,1)), nn.Flatten())

@d2l.add_to_class(GoogleNet)
def b0(self):
    return nn.BatchNorm2d(6);



@d2l.add_to_class(GoogleNet)
def __init__(self, lr=0.1, num_classes=10):
    #super(GoogleNet, self).__init__()
    
    super(GoogleNet, self).__init__('l1')        
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b0(), self.b1(), self.b2(), self.b3(), self.b4(),
                             self.b5(), nn.LazyLinear(1))
    self.net.apply(d2l.init_cnn)

In [ ]:
google_net = GoogleNet(lr = 0.1)
X = df2t2[0][0]
ml_utils.layer_summary(google_net, X.shape)
#ml_utils.parameter_summary(google_net)

In [ ]:
trainer = modules.Trainer(max_epochs=15, num_gpus=1)
with Timer('training'):
    trainer.fit(google_net, dm)

In [ ]:
eval_init_loss(google_net, dm)

In [ ]:
dataset = dm.get_dataloader(train = True).dataset

In [ ]:
n = len(dataset)

In [ ]:
y = np.empty(n)
for i in np.arange(n):
    y[i] = dataset[i][1][0]

In [ ]:
np.abs(y).sum()/1e6

In [ ]:
net_cpu = google_net.to('cpu')

In [ ]:
y_hat = np.empty(n)
for i in np.arange(n):
    x = dataset[i][0]
    y_hat[i] = net_cpu(x.unsqueeze(0))

In [ ]:
x = dataset[0:2][0]

In [ ]:
i

In [ ]:
len(dataset)

In [ ]:
google_net = google_net.to('cuda:0')

In [ ]:
dl = dm.get_dataloader(True)
y_hat_list = []
yw_list = []
for i, batch in enumerate(dl):    
    X, YW = batch[0], batch[1]
    ZEROS = torch.zeros(YW.shape[0])
    with torch.no_grad():            
        #init_loss += net.loss(ZEROS, YW)
        Y_hat = google_net(X.to(ml_utils.gpu()))
        y_hat_list.append(Y_hat.to('cpu'))
        yw_list.append(YW)        
        print(Y_hat)
        #loss += net.loss(Y_hat, YW.to(ml_utils.gpu()))

In [ ]:
loss = 0.0
loss_y = 0.0
loss_y_hat = 0.0
for y_hat, yw in zip(y_hat_list, yw_list):
    y_hat = y_hat.to('cpu')
    y = yw[:,0]
    loss += np.mean(np.abs(y_hat - y))
    loss_y += np.mean(np.abs(y))
    loss_y_hat += np.mean(np.abs(y_hat))

In [ ]:
y

In [ ]:
y_hat.view(-1)

In [ ]:
y_hat_list[3]

In [ ]:
yw_list

In [ ]:
y = [yw[:,0] for yw in yw_list]

In [ ]:
y

In [ ]:
concatenated_tensor = torch.cat(y, dim=0)

In [ ]:
y = concatenated_tensor

In [ ]:
y.abs().sum()

In [ ]:
y.std()

In [ ]:
y.mean()

In [ ]:
y_hat = [y_hat.view(-1) for y_hat in y_hat_list]

In [ ]:
y_hat = torch.cat(y_hat, dim=0)